# Tutorial1_Extension

Here I try to explore some of the concepts that I did not understand very well from the tutorial.


# Libraries

In [3]:
# mathematical
import numpy as np

# plotting
import matplotlib.pyplot as plt

# Covariance Matrix

My understanding of the covariance matrix is very much based on stats, where:

$$\mathrm{Cov}(X,Y) = E\left [ \left(X-\mu_X\right) \left(Y-\mu_Y\right)\right ] = E\left [XY\right]-\mu_X\mu_Y$$

Now, if we were to generalise this for a matrix $\underline{\underline{X}}$ of predictors, where the $i$th index represents the datapoint (data index), and the $j$th index represents the feature (or predictor), we would have something like this:

$$\Sigma_{ij}= \frac{1}{n}X_{ki} X_{kj} - \mu_{ij}$$

Where the summation convention from index notation (Computational Continuum Mechanics) is applied.

However, we got a very odd definition in the machine learning course, which I still don't fully comprehend. This is, as per Dr Huthwaites notes:

$$C_{ij} = (X-\mu)_{ki}(X-\mu)_{kj}$$
Let's explore this.

His equation does seem very similar to the non-simplified one that we wrote in the beginning, meaning it is something like this:

$$\Sigma_{ij} = E\left[C_{ij}\right]$$

Let's explore this further

In [18]:
# my understanding

def covariance_matrix(X):
    """
    Calculates the covariance matrix for a given input X with n = X.shape[1] predictors
    """
    n = X.shape[1]
    n_points = X.shape[0]
    Sigma = np.zeros([n,n])
    mu = [X[:,i].mean(axis = 0) for i in range(n)]
    print(mu)
    for i in range(n): # rows
        for j in range(i,n):
            Sigma[i,j] = (X[:,i]*X[:,j]).sum(axis = 0)/n_points - mu[i]*mu[j]
            if i != j:
                Sigma[j,i] = Sigma[i,j]
            
    return Sigma

X = np.array(
    [[1,2],
     [3,4],
     [5,7]]
)    

Sigma = covariance_matrix(X)
Sigma

[3.0, 4.333333333333333]


array([[2.66666667, 3.33333333],
       [3.33333333, 4.22222222]])

In [27]:
def huthwaite_cov(X):
    """
    Calculates the huthwaite covariance matrix
    """
    n = X.shape[1]
    n_points = X.shape[0]
    C = np.zeros([n,n])
    mu = [X[:,i].mean(axis = 0) for i in range(n)]
    print(mu)
    for i in range(n): # rows
        for j in range(i,n):
            C[i,j] = ((X[:,i]-mu[i])*(X[:,j]-mu[j])).sum(axis = 0)
            if i != j:
                C[j,i] = C[i,j]
            
    return C

def covariance_matrix(C):
    """
    Calculates the covariance matrix from the huthwaite_matrix C
    """
    pass

# not quite convinced?

X = np.array(
    [[1,2],
     [3,4],
     [5,7]]
)    

C = huthwaite_cov(X)

Sigma = C/3
Sigma

[3.0, 4.333333333333333]


array([[2.66666667, 3.33333333],
       [3.33333333, 4.22222222]])

It seems that our understanding of Huthwaite's C function was in fact correct. It's quite odd that he would use such a definition, perhaps it's worth discussing this with him?

The tutorial included another covariance matrix, albeit this was a bit, confusing?
It did not make sense to me. So it's worth exploring this for a bit.

In [113]:
def get_cov(*sigma, **kwargs):
    """
    Creates a covariance matrix based on the value of theta 
    
    Attributes:
    -----------
    
    *sigma: *int
        Can be 2 or 3 dimensions, defines the standard deviations for each degree of freedom
        
    **kwargs: *keyword = *int
        kwargs for the create_rotation_matrix function, defines the intensity of the covariance
        
    Notes:
    ------
    - add functionality for N dimension sigma (in that case, the concept of the rotation matrix would cease
    to exist though)
    
    """
    dof = len(sigma)
    covar = np.zeros([dof,dof])
    for i,std in enumerate(sigma):
        covar[i,i] = std
    
    kwargs.update((x, y / 360 * 2 * np.pi) for x, y in kwargs.items()) # convert angles to radians
    R = create_rotation_matrix(**kwargs)
    try:
        covar = np.matmul(np.matmul(R, covar), R.T)
    except:
        raise ValueError((
            'Your covariance matrix is expected to be of shape {}, whereas your rotation matrix is of'
            'shape {}. Check that you are not trying to rotate a 2D matrix in 3 dimensions'
        ).format(covar.shape,R.shape))

    return covar

def create_rotation_matrix(theta_x = 0,theta_y = 0, theta_z = 0):
    """
    creates the 2D or 3D rotation matrices depending on the inputs that are given. Note that all angles must
    in RADIANS
    
    Attributes:
    -----------
    
    theta_x: int
        defines the yaw of the system
    
    theta_y: int
        defines the pitch of the system
    
    theta_z: int
        defines the roll of the system
    """
    
    if theta_y or theta_z:
        R_x = np.array([
            [1, 0, 0],
            [0, np.cos(theta_x), -np.sin(theta_x)],
            [0, np.sin(theta_x), np.cos(theta_x)]
        ])
        
        R_y = np.array([
            [np.cos(theta_y), 0, np.sin(theta_y)],
            [0, 1, 0],
            [-np.sin(theta_y),0, np.cos(theta_y)]
        ])
        
        R_z = np.array([
            [np.cos(theta_z), -np.sin(theta_z), 0],
            [np.sin(theta_z), np.cos(theta_z), 0],
            [0,0,1]
        ])
        R = np.matmul(np.matmul(R_x,R_y),R_z)
        
    else:
        R = np.array([
            [np.cos(theta_x), -np.sin(theta_x)],
            [np.sin(theta_x), np.cos(theta_x)]
        ])
        
    return R


# create a function that can plot the covariances for multiple plots, i.e. 2D 3D as a function of the theta
# that you give it (maybe it's best to convert what you have above into a CLASS for ease?)

After exploration, it seems to me that Huthwaite's get_cov function is just a makeshift method for creating a covariance matrix...

This is because after multiplication, you are left with the following:


$$\mathrm{Cov} = \left [\begin{matrix}
\sigma_x^2 & \sigma_x\cos\theta\sin\theta - \sigma_y\cos\theta\sin\theta \\
\sigma_x\cos\theta\sin\theta - \sigma_y\cos\theta\sin\theta  & \sigma_y^2
\end{matrix}\right]$$

Theta thus controls the degree to which these points are related.

# Meshgrid